In [10]:
import numpy as np
import wave

def IAV(data,frame_size):
    value=0.0;

    for i in range (frame_size):
        value += abs(data[i])
    
    return value

def getIAVThreshold(y,length,frame_size):
    IAV_data_size= 0
    Max_length = 0
    Min_length = 0
    threshold = 0.0
    Max_avg = 0.0
    Min_avg = 0.0
    
    IAV_data_size = int(length / frame_size)
    Max_length = int(IAV_data_size * 0.2)
    Min_length = int(IAV_data_size * 0.2)
    
    IAV_data = np.zeros(IAV_data_size, dtype=np.float32)
    Max = np.zeros(Max_length, dtype=np.float32)
    Min = np.zeros(Min_length, dtype=np.float32)
    
    j = 0
    
    for i in range(0,length,frame_size):
        if (j >= IAV_data_size):
            break
            
        data_sub = y[i:i+frame_size]
        IAV_data[j] = IAV(data_sub,frame_size)
        j = j + 1
        
    sort_indices = np.argsort(IAV_data)[::-1]
    IAV_data[:] = IAV_data[sort_indices]
        
    for i in range(Max_length):
        Max[i] = IAV_data[i]
        
    IAV_data.sort()
    
    for i in range(Min_length):
        Min[i] = IAV_data[i]
                          
    threshold = 0.0
    
    Max_avg = np.mean(Max)
    Min_avg = np.mean(Min)
    
    if(Max_avg*0.7 > Min_avg):
        threshold = (Max_avg - Min_avg) * 0.1
        threshold = threshold + Min_avg        
    else:
        threshold = Max_avg*0.2
        
    return threshold

def getStartPointEndPoint(y,sr,length,Window_size):
    start_point = 0
    end_point = 0
    
    IAV_th = getIAVThreshold(y,length,Window_size)
    th = IAV_th/Window_size*2
    
    # start point
    for i in range(0,length-1,int(Window_size/3)):
        iav = IAV(y[i:i+Window_size],Window_size)
        
        if iav > IAV_th:
            start_point = i 
            for j in range(Window_size):
                if (abs(y[i+j]) > th):
                    start_point = i+j
                    break
            break
    
    
    if (start_point - Window_size > 0) == 1:
        start_point = start_point - Window_size
    
    # end point
    for i in range(length-1,0,-int(Window_size/3)):
        iav = IAV(y[i-Window_size:i],Window_size)
        
        if iav > IAV_th:
            end_point = i
            for j in range(Window_size):
                if (abs(y[i-j]) > th):
                    end_point = i-j
                    break
            break
    
    
    if (end_point + Window_size < length) == 1:
        end_point = end_point + Window_size
    
    return start_point, end_point

def load_wav(filename,sr):
    waveFile = wave.open(filename,'rb')
    nchannel = waveFile.getnchannels()
    length = waveFile.getnframes()
    nBytes = nchannel * length
    format = "<" +str(nBytes) + "h"

    waveData = waveFile.readframes(length)
    data_a = struct.unpack(format,waveData)

    waveFile.close()

    wave_channel1 =list(data_a)
    y = np.array(wave_channel1,np.float32)
    return y,sr

In [ ]:
path = "m2(1).wav"
sr = 16000

wav = load_wav(path, sr=sr)

Window_size = 500

length = len(wav)

start_point, end_point = getStartPointEndPoint(wav,sr,length,Window_size)

y = wav[start_point:end_point]

In [ ]:
plt.plot(y)
plt.show()